In [1]:
from dataclasses import dataclass
import pandas as pd

In [2]:
@dataclass(frozen=True)
class TestData:
    value: float
    metric_type: str

    def __str__(self):
        return "Metric {0}: value -> {{{1}}}".format(self.metric_type, self.value)

In [3]:
def collect_data(sources):
    data = []
    for source in sources:
        if source.type == 'test':
            data.extend([acc for acc in source.results if acc.metric_type == 'accuracy'])
        elif source.type == 'project':
            data.extend([spd for spd in source.outputs if spd.metric_type == 'completion_time'])
    return data

In [4]:
def normalize_data(raw_data: [TestData]):
    normalized = []
    max_time = 10000  # Default max time for normalization
    for item in raw_data:
        if item.metric_type == 'accuracy':
            normalized.append({'accuracy': item.value / 100})
        elif item.metric_type == 'completion_time':
            normalized.append({'completion_time': 1 - (item.value / max_time)})
    return normalized

In [5]:
def calculate_skill_level(normalized_data: list[object]):    
    metrics = {'accuracy': [], 'speed': []}
    
    for item in normalized_data:        
        if item.get('accuracy') is not None:
            metrics['accuracy'].append(item.get('accuracy'))
        elif item.get('completion_time') is not None:
            metrics['speed'].append(item.get('completion_time'))
    
    df_accuracy = pd.Series(metrics['accuracy'])
    df_speed = pd.Series(metrics['speed'])

    avg_accuracy = 0
    avg_speed = 0
    
    if len(df_accuracy) > 0:
        avg_accuracy = df_accuracy.sum() / len(df_accuracy)
        print("Added to average accuracy, NEW: {0} => avg_accuracy == {1}".format(df_accuracy.sum(), avg_accuracy))
    if len(df_speed) > 0:
        avg_speed = df_speed.sum() / len(df_speed)
        print("Added to average speed, NEW: {0} => avg_speed == {1}".format(df_speed.sum(), avg_speed))
    
    return (avg_accuracy * 0.7) + (avg_speed * 0.3)

In [6]:
def assign_level(score):
    if score < 0.5:
        return "Novice"
    elif score < 0.8:
        return "Intermediate"
    else:
        return "Advanced"

In [7]:
# Sample usage - performance analysis pipeline

# Create some test data
test1 = TestData(value=75, metric_type='accuracy')
test2 = TestData(value=82, metric_type='accuracy')
test3 = TestData(value=90, metric_type='accuracy')

project1 = TestData(value=3600, metric_type='completion_time')  # 1 hour
project2 = TestData(value=7200, metric_type='completion_time')  # 2 hours

sources = [
    type('test_source', (), {'type': 'test', 'results': {test1, test2, test3}})(),
    type('project_source', (), {'type': 'project', 'outputs': {project1, project2}})()
]

# Collect and process data
raw_data = collect_data(sources)
print(f"Raw Data Collected: {raw_data}")

normalized = normalize_data(raw_data)
print(f"Normalized Data: {normalized}")

skill_score = calculate_skill_level(normalized)
print(f"Skill Score: {skill_score:.8f}")

level = assign_level(skill_score)
print(f"Assigned Level: {level}")    

Raw Data Collected: [TestData(value=75, metric_type='accuracy'), TestData(value=90, metric_type='accuracy'), TestData(value=82, metric_type='accuracy'), TestData(value=3600, metric_type='completion_time'), TestData(value=7200, metric_type='completion_time')]
Normalized Data: [{'accuracy': 0.75}, {'accuracy': 0.9}, {'accuracy': 0.82}, {'completion_time': 0.64}, {'completion_time': 0.28}]
Added to average accuracy, NEW: 2.4699999999999998 => avg_accuracy == 0.8233333333333333
Added to average speed, NEW: 0.92 => avg_speed == 0.46
Skill Score: 0.71433333
Assigned Level: Intermediate
